## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-03-14-51-34 +0000,nyt,Storm Floris Is Set to Hit Britain With Powerf...,https://www.nytimes.com/2025/08/03/weather/sto...
1,2025-08-03-14-48-54 +0000,bbc,Car finance mis-selling payout scheme could co...,https://www.bbc.com/news/articles/cgjy29zql25o...
2,2025-08-03-14-47-57 +0000,nypost,Fundraiser for Virginia Councilman Lee Vogler ...,https://nypost.com/2025/08/03/us-news/virginia...
3,2025-08-03-14-43-58 +0000,nyt,Putin Widens Effort to Control Russia’s Internet,https://www.nytimes.com/2025/08/03/world/europ...
4,2025-08-03-14-40-27 +0000,nyt,"Trump Fires Official Over Jobs Report, Echoing...",https://www.nytimes.com/2025/08/03/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,26
23,attack,8
89,fire,8
132,new,8
91,oil,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
120,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...,82
75,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...,54
106,2025-08-03-00-48-00 +0000,wsj,Erika McEntarfer led the Bureau of Labor Stati...,https://www.wsj.com/politics/policy/the-econom...,53
162,2025-08-02-15-31-46 +0000,nypost,Trump demurs on pardoning disgraced former Rep...,https://nypost.com/2025/08/02/us-news/presiden...,51
96,2025-08-03-02-53-15 +0000,nypost,Trump tells Chuck Schumer to ‘GO TO HELL’ over...,https://nypost.com/2025/08/02/us-news/trump-te...,50


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
120,82,2025-08-02-22-28-00 +0000,wsj,A watchdog agency opened an investigation into...,https://www.wsj.com/politics/policy/watchdog-a...
36,49,2025-08-03-10-34-36 +0000,nypost,Ukrainian drone attack sparks massive fire at ...,https://nypost.com/2025/08/03/world-news/ukrai...
75,48,2025-08-03-07-39-06 +0000,nypost,US envoy tells Israeli hostage families he’s w...,https://nypost.com/2025/08/03/world-news/us-en...
111,35,2025-08-03-00-04-44 +0000,nypost,Cincinnati police chief under intense scrutiny...,https://nypost.com/2025/08/02/us-news/cincinna...
131,32,2025-08-02-20-20-05 +0000,nypost,Bill Maher slams woke mob over Sydney Sweeney ...,https://nypost.com/2025/08/02/us-news/bill-mah...
56,32,2025-08-03-09-30-00 +0000,wsj,"Like Zohran Mamdani in New York, the rise of M...",https://www.wsj.com/politics/elections/sociali...
138,32,2025-08-02-19-21-35 +0000,nypost,Three killed in Russia as Ukraine takes aim at...,https://nypost.com/2025/08/02/world-news/three...
89,26,2025-08-03-04-27-43 +0000,bbc,"Their children can't eat, speak or walk - so f...",https://www.bbc.com/news/articles/c39d0zj0110o...
26,24,2025-08-03-12-00-00 +0000,nypost,Elon Musk’s long history of having a short fus...,https://nypost.com/2025/08/03/us-news/elon-mus...
74,24,2025-08-03-07-55-15 +0000,nypost,Aussie running for Boston suburb mayor says th...,https://nypost.com/2025/08/03/us-news/massachu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
